In [7]:
import os, sys
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))

from src.configuration.constants import PROCESSED_DATA_DIRECTORY, ROOT_DIRECTORY, INTERIM_DATA_DIRECTORY, LOGS_DATA_DIRECTORY
from src.utils.dataset import load_dataset


import collections
import logging
import os
import pathlib
import re
import string
import sys
import time
import math
import tempfile
from functools import partial

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import pandas as pd

from orion.data import load_signal
from orion import Orion
from orion.data import load_anomalies

import tensorflow as tf
from functools import partial
from mlprimitives import load_primitive

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from orion.evaluation.contextual import contextual_f1_score
import pandas as pd
import random


from mlprimitives import load_primitive

logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

# Positional encoding

In [8]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [9]:
# Test Example
n, d = 100, 25
pos_encoding = positional_encoding(n, d)
print(pos_encoding.shape)
pos_encoding = pos_encoding[0]

(1, 100, 25)


# Masking

In [10]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

In [11]:
# 0 are pads
x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])
create_padding_mask(x)

<tf.Tensor 'strided_slice_3:0' shape=(3, 1, 1, 5) dtype=float32>

In [12]:
# look-ahead mask
x = tf.random.uniform((1, 3))
temp = create_look_ahead_mask(x.shape[1])
temp

<tf.Tensor 'sub_1:0' shape=(3, 3) dtype=float32>

# Multihead Attention

In [13]:
from src.models.layers import MultiHeadAttention

temp_mha = MultiHeadAttention(d_model=25, num_heads=5)
y = tf.random.uniform((1, 100, 25))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, k=y, q=y, mask=None)
out.shape, attn.shape

(TensorShape([Dimension(1), Dimension(100), Dimension(25)]),
 TensorShape([Dimension(1), Dimension(5), Dimension(100), Dimension(100)]))

# Point wise feed forward network

In [14]:
from src.models.layers import point_wise_feed_forward_network

sample_ffn = point_wise_feed_forward_network(512, 2048)
sample_ffn(tf.random.uniform((64, 50, 512))).shape

TensorShape([Dimension(64), Dimension(50), Dimension(512)])

# Encoder

In [15]:
from src.models.layers import EncoderLayer

sample_encoder_layer = EncoderLayer(512, 8, 2048)

sample_encoder_layer_output = sample_encoder_layer(
    tf.random.uniform((64, 43, 512)), False, None)

sample_encoder_layer_output.shape  # (batch_size, input_seq_len, d_model)

TensorShape([Dimension(64), Dimension(43), Dimension(512)])

In [16]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    # self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding,
                                            self.d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embedding and position encoding.
    # x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]
   

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x  # (batch_size, input_seq_len, d_model)

In [17]:
sample_encoder = Encoder(num_layers=2, d_model=25, num_heads=5,
                         dff=1, input_vocab_size=None,
                         maximum_position_encoding=10000)
temp_input = tf.random.uniform((1, 100, 25), dtype=tf.float32, minval=0, maxval=200)

sample_encoder_output = sample_encoder(temp_input, training=False, mask=None)

print(sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)

(1, 100, 25)


In [18]:
sample_encoder.trainable

True

# Decoder

In [19]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

In [20]:
sample_decoder_layer = DecoderLayer(512, 8, 2048)

sample_decoder_layer_output, _, _ = sample_decoder_layer(
    tf.random.uniform((64, 50, 512)), sample_encoder_layer_output,
    False, None, None)

sample_decoder_layer_output.shape  # (batch_size, target_seq_len, d_model)

TensorShape([Dimension(64), Dimension(50), Dimension(512)])

In [21]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [22]:
sample_decoder = Decoder(num_layers=2, d_model=512, num_heads=8,
                         dff=2048, target_vocab_size=8000,
                         maximum_position_encoding=5000)
temp_input = tf.random.uniform((64, 26), dtype=tf.float32, minval=0, maxval=200)

output, attn = sample_decoder(temp_input,
                              enc_output=sample_encoder_output,
                              training=False,
                              look_ahead_mask=None,
                              padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape

ValueError: in converted code:

    /var/folders/_5/f7yzqnxs6694l3c_yk9mjpzr0000gn/T/ipykernel_85722/955386546.py:29 call  *
        x, block1, block2 = self.dec_layers[i](x, enc_output, training,
    /Users/lcwong/opt/anaconda3/envs/hitlads-env/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:854 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /var/folders/_5/f7yzqnxs6694l3c_yk9mjpzr0000gn/T/ipykernel_85722/1969818830.py:26 call  *
        attn2, attn_weights_block2 = self.mha2(
    /Users/lcwong/opt/anaconda3/envs/hitlads-env/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:854 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /Users/lcwong/PycharmProjects/hitlads/src/models/layers/transformer.py:103 call  *
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    /Users/lcwong/PycharmProjects/hitlads/src/models/layers/transformer.py:92 split_heads  *


# Transformer

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, pe_input, pe_target, rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, pe_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                           target_vocab_size, pe_target, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs, training):
    # Keras models prefer if you pass all your inputs in the first argument
    inp, tar = inputs

    enc_padding_mask, look_ahead_mask, dec_padding_mask = self.create_masks(inp, tar)

    enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)

    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

    return final_output, attention_weights

  def create_masks(self, inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    look_ahead_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, look_ahead_mask, dec_padding_mask

In [ ]:
sample_transformer = Transformer(
    num_layers=2, d_model=512, num_heads=8, dff=2048,
    input_vocab_size=8500, target_vocab_size=8000,
    pe_input=10000, pe_target=6000)

temp_input = tf.random.uniform((64, 38), dtype=tf.float32, minval=0, maxval=200)
temp_target = tf.random.uniform((64, 36), dtype=tf.float32, minval=0, maxval=200)

fn_out, _ = sample_transformer([temp_input, temp_target], training=False)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)

# Attention TadGAN

## Dataset

### Loading

In [23]:
dataset = load_dataset('interim', 'MSL')

train_split = dataset['train']
test_split = dataset['test']
anomalies_split = dataset['anomaly']

signal_to_dataset = {}
for signal in train_split.signal.unique():
    signal_to_dataset[signal] = {
        'train': train_split[train_split.signal == signal],
        'test': test_split[test_split.signal == signal],
        'anomaly': anomalies_split[anomalies_split.signal == signal],
    }

signal_to_dataset.keys()

dict_keys(['M-1', 'M-2', 'M-3', 'M-7', 'T-5', 'T-4', 'M-6', 'M-4', 'M-5', 'F-8', 'S-2', 'D-15', 'P-14', 'P-15', 'D-14', 'D-16', 'F-7', 'P-11', 'F-5', 'F-4', 'P-10', 'T-13', 'T-12', 'T-9', 'T-8', 'C-1', 'C-2'])

In [24]:
m2_dataset = signal_to_dataset['M-2']

train = m2_dataset['train']
test = m2_dataset['test']
anomalies = m2_dataset['anomaly']

index_train = train['index'].astype(int)
X_train = train[['anomaly'] + list(train.columns)[5:]]

index_test = test['index'].astype(int).reset_index(drop=True)
X_test = test[['anomaly'] + list(test.columns)[5:]]

### Pre-Processing

In [25]:
def fit_processing(X, index):
    primitives = []
    
    
    # This primitive is an imputation transformer for filling missing values
    params = {
        'X': X
    }
    primitive = load_primitive('sklearn.impute.SimpleImputer', arguments=params)
    primitive.fit()
    primitives.append(primitive)
    X = primitive.produce(X=X)
    print(primitive, X.shape)
    
    # This primitive transforms features by scaling each feature to a given range
    params = {
        "feature_range": [-1, 1], 
        'X': X,
    }
    primitive = load_primitive('sklearn.preprocessing.MinMaxScaler', arguments=params)
    primitive.fit()
    primitives.append(primitive)
    X = primitive.produce(X=X)
    print(primitive, X.shape)
    
    # Uses a rolling window approach to create the sub-sequences out of time series data
    params = {
        "target_column": 0, 
        "window_size": 100, 
        'target_size': 1, 
        'step_size': 1
    }
    primitive = load_primitive('mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences',
                               arguments=params)
    primitives.append(primitive)
    X, y, index, target_index = primitive.produce(X=X, index=index)

    # Target / target size is the next interval that is trying to predict.
    # Index is the start of the interval
    print(primitive, X.shape, y.shape, index.shape, target_index.shape)
    
    return X, index, primitives 

In [26]:
X, index, primitives = fit_processing(X_train, index_train)

MLBlock - sklearn.impute.SimpleImputer (2208, 56)
MLBlock - sklearn.preprocessing.MinMaxScaler (2208, 56)
MLBlock - mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences (2108, 100, 56) (2108, 1) (2108,) (2108,)


In [27]:
X_train = X[:, :, 1:]
y_train = np.expand_dims(X_train[:, :, 0], 2)
index_train = index

X_train.shape, y_train.shape, index_train.shape

((2108, 100, 55), (2108, 100, 1), (2108,))

In [28]:
def produce_processing(X, index, primitives):
    X = primitives[0].produce(X=X)
    X = primitives[1].produce(X=X)
    X, y, index, target_index = primitives[2].produce(X=X, index=index)
    return X, index

In [29]:
X, index = produce_processing(X_test, index_test, primitives)

In [30]:
X_test = X[:, :, 1:]
y_test = np.expand_dims(X_test[:, :, 0], 2)
index_test = index
labels_test = np.array([1 if sum(i) > 0 else 0 for i in X[:, :, 0]])


X_test.shape, y_test.shape, index_test.shape, labels_test.shape

((2177, 100, 55), (2177, 100, 1), (2177,), (2177,))

## Model

In [55]:
input_shape=X_train[0].shape
target_shape=y_train[0].shape
latent_dim=20
learning_rate=0.0005
epochs=70
batch_size=64
iterations_critic=5
latent_shape = (latent_dim, 1)

shape = np.asarray(X_train)[0].shape
length = shape[0]
target_shape = np.asarray(y_train)[0].shape


generator_reshape_dim = length // 2
generator_reshape_shape = (length // 2, 1)
encoder_reshape_shape = latent_shape

encoder_input_shape = shape
generator_input_shape = latent_shape
critic_x_input_shape = target_shape
critic_z_input_shape = latent_shape

lstm_units = 100
dense_units = 20

In [90]:
def build_encoder(input_shape, lstm_units, dense_units, encoder_reshape_shape):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    # model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=lstm_units, return_sequences=True)))
    model.add(Encoder(num_layers=2, d_model=input_shape[-1], num_heads=5,
                      dff=1, input_vocab_size=None,
                      maximum_position_encoding=10000))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=dense_units))
    model.add(tf.keras.layers.Reshape(target_shape=encoder_reshape_shape))
    return tf.keras.Model(x, model(x))
              
encoder = build_encoder(
    input_shape=encoder_input_shape,
    lstm_units=lstm_units,
    dense_units=dense_units,
    encoder_reshape_shape=encoder_reshape_shape,
)
encoder.summary()

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 100, 55)]         0         
_________________________________________________________________
sequential_20 (Sequential)   (None, 20, 1)             135432    
Total params: 135,432
Trainable params: 135,432
Non-trainable params: 0
_________________________________________________________________


In [91]:
def build_generator(input_shape, generator_reshape_dim, generator_reshape_shape):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=generator_reshape_dim))
    model.add(tf.keras.layers.Reshape(target_shape=generator_reshape_shape))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    
    model.add(tf.keras.layers.UpSampling1D(size=2))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=1)))
    model.add(tf.keras.layers.Activation(activation='tanh'))
    return tf.keras.Model(x, model(x))
              
generator = build_generator(
    input_shape=generator_input_shape,
    generator_reshape_dim=generator_reshape_dim,
    generator_reshape_shape=generator_reshape_shape,
)
generator.summary()

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 20, 1)]           0         
_________________________________________________________________
sequential_23 (Sequential)   (None, 100, 1)            133787    
Total params: 133,787
Trainable params: 133,787
Non-trainable params: 0
_________________________________________________________________


In [92]:
def build_critic_x(input_shape):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=1))
    return tf.keras.Model(x, model(x))
              
critic_x = build_critic_x(
    input_shape=critic_x_input_shape
)
critic_x.summary()

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        [(None, 100, 1)]          0         
_________________________________________________________________
sequential_24 (Sequential)   (None, 1)                 67393     
Total params: 67,393
Trainable params: 67,393
Non-trainable params: 0
_________________________________________________________________


In [93]:
def build_critic_z(input_shape, dense_units=20):
    x = tf.keras.Input(shape=input_shape)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=dense_units))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Dense(units=dense_units))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Dense(units=1))
    return tf.keras.Model(x, model(x))
              
critic_z = build_critic_z(
    input_shape=critic_z_input_shape,
)
critic_z.summary()

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        [(None, 20, 1)]           0         
_________________________________________________________________
sequential_25 (Sequential)   (None, 1)                 861       
Total params: 861
Trainable params: 861
Non-trainable params: 0
_________________________________________________________________


In [94]:
def _wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

def _gradient_penalty_loss(y_true, y_pred, averaged_samples):
    gradients = K.gradients(y_pred, averaged_samples)[0]
    gradients_sqr = K.square(gradients)
    gradients_sqr_sum = K.sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
    gradient_l2_norm = K.sqrt(gradients_sqr_sum)
    gradient_penalty = K.square(1 - gradient_l2_norm)
    return K.mean(gradient_penalty)

In [95]:
optimizer = tf.keras.optimizers.Adam(learning_rate)

In [96]:
generator.trainable = False
encoder.trainable = False

x = tf.keras.Input(shape=input_shape)
y = tf.keras.Input(shape=target_shape)
z = tf.keras.Input(shape=(latent_dim, 1))

x_ = generator(z)
z_ = encoder(x)
fake_x = critic_x(x_) # Fake
valid_x = critic_x(y) # Truth

In [97]:
alpha = K.random_uniform((64, 1, 1))
interpolated_x = (alpha * [y, x_][0]) + ((1 - alpha) * [y, x_][1])
validity_interpolated_x = critic_x(interpolated_x)
partial_gp_loss_x = partial(_gradient_penalty_loss, averaged_samples=interpolated_x)
partial_gp_loss_x.__name__ = 'gradient_penalty'
critic_x_model = tf.keras.Model(inputs=[y, z], outputs=[valid_x, fake_x, validity_interpolated_x])
critic_x_model.compile(loss=[_wasserstein_loss, _wasserstein_loss, partial_gp_loss_x], 
                       optimizer=optimizer, loss_weights=[1, 1, 10])

In [98]:
fake_z = critic_z(z_)
valid_z = critic_z(z)
alpha = K.random_uniform((64, 1, 1))
interpolated_z = (alpha * [z, z_][0]) + ((1 - alpha) * [z, z_][1])
validity_interpolated_z = critic_z(interpolated_z)
partial_gp_loss_z = partial(_gradient_penalty_loss, averaged_samples=interpolated_z)
partial_gp_loss_z.__name__ = 'gradient_penalty'
critic_z_model = tf.keras.Model(inputs=[x, z], outputs=[valid_z, fake_z,validity_interpolated_z])
critic_z_model.compile(loss=[_wasserstein_loss, _wasserstein_loss,
                                  partial_gp_loss_z], optimizer=optimizer,
                            loss_weights=[1, 1, 10])

In [99]:
critic_x.trainable = False
critic_z.trainable = False
generator.trainable = True
encoder.trainable = True

z_gen = tf.keras.Input(shape=(latent_dim, 1))
x_gen_ = generator(z_gen)
x_gen = tf.keras.Input(shape=input_shape)
z_gen_ = encoder(x_gen)
x_gen_rec = generator(z_gen_)
fake_gen_x = critic_x(x_gen_)
fake_gen_z = critic_z(z_gen_)

encoder_generator_model = tf.keras.Model([x_gen, z_gen], [fake_gen_x, fake_gen_z, x_gen_rec])
encoder_generator_model.compile(loss=[_wasserstein_loss, _wasserstein_loss, 'mse'], 
                                optimizer=optimizer,
                                loss_weights=[1, 1, 10])

### Training

https://gist.github.com/erenon/91f526302cd8e9d21b73f24c0f9c4bb8

In [100]:
fake = np.ones((batch_size, 1))
valid = -np.ones((batch_size, 1))
delta = np.ones((batch_size, 1))

indices = np.arange(X_train.shape[0])

epoch_loss = []

for epoch in range(1, epochs + 1):
    np.random.shuffle(indices)
    X_ = X_train[indices]
    y_ = y_train[indices]

    epoch_g_loss = []
    epoch_cx_loss = []
    epoch_cz_loss = []

    minibatches_size = batch_size * iterations_critic
    num_minibatches = int(X_.shape[0] // minibatches_size)

    for i in range(num_minibatches):
        minibatch = X_[i * minibatches_size: (i + 1) * minibatches_size]
        y_minibatch = y_[i * minibatches_size: (i + 1) * minibatches_size]

        for j in range(iterations_critic):
            x = minibatch[j * batch_size: (j + 1) * batch_size]
            y = y_minibatch[j * batch_size: (j + 1) * batch_size]
            z = np.random.normal(size=(batch_size, latent_dim, 1))
            epoch_cx_loss.append(
                critic_x_model.train_on_batch([y, z], [valid, fake, delta]))
            epoch_cz_loss.append(
                critic_z_model.train_on_batch([x, z], [valid, fake, delta]))
        
        epoch_g_loss.append(
            encoder_generator_model.train_on_batch([x, z], [valid, valid, y]))
        
    cx_loss = np.mean(np.array(epoch_cx_loss), axis=0)
    cz_loss = np.mean(np.array(epoch_cz_loss), axis=0)
    g_loss = np.mean(np.array(epoch_g_loss), axis=0)
    
    epoch_loss.append([cx_loss, cz_loss, g_loss])
    
    print('Epoch: {}/{}, [Dx loss: {}] [Dz loss: {}] [G loss: {}]'.format(
        epoch, epochs, cx_loss, cz_loss, g_loss))
    

Epoch: 1/70, [Dx loss: [-2.0129554  -5.597753    0.77514684  0.280965  ]] [Dz loss: [ 3.8211598  -0.2732207   2.6121001   0.14822808]] [G loss: [ 0.4014206  -0.91131824 -2.1429665   0.34557056]]
Epoch: 2/70, [Dx loss: [-5.066792   -9.203266    3.4531922   0.06832839]] [Dz loss: [ 4.151757   -0.18826985  3.4075956   0.09324302]] [G loss: [-2.2937276  -3.3622925  -0.48067975  0.15492444]]
Epoch: 3/70, [Dx loss: [-0.18571615 -7.890046    7.4959264   0.02084036]] [Dz loss: [7.1281824e+00 1.4005139e-03 6.2706766e+00 8.5610501e-02]] [G loss: [-7.0533967  -7.244167   -1.5558213   0.17465913]]
Epoch: 4/70, [Dx loss: [-1.6173195   5.9525776  -8.080727    0.05108303]] [Dz loss: [2.3114508e+01 1.7453984e-03 2.2085546e+01 1.0272168e-01]] [G loss: [ -3.401511    10.515015   -15.674396     0.17578687]]
Epoch: 5/70, [Dx loss: [ -3.3102913    7.805539   -11.512557     0.03967267]] [Dz loss: [52.43507     0.1650541  51.002308    0.12677048]] [G loss: [-32.916107   10.031751  -44.85469     0.1906832]]
E

In [101]:
[Dx loss: [ -1.239893   -21.721457    20.22298      0.02585853]] 
[Dz loss: [12.848619   2.4626553 -1.6048353  1.19908  ]] 
[G loss: [-17.0163     -19.386038     1.6985518    0.06711876]]

SyntaxError: invalid syntax (1572066141.py, line 1)

In [ ]:
[Dx loss: [-0.23936272  9.642991   -9.992036    0.01096798]] 
[Dz loss: [ 2.4516199   3.1085587  -3.3864095   0.27294713]] 
[G loss: [13.0496435   8.32087     3.5459964   0.11827757]]

## Evaluation

In [73]:
z_ = encoder.predict(X_test)
y_hat = generator.predict(z_)
critic = critic_x.predict(y_test)

In [74]:
# computes an array of anomaly scores based on a combination of reconstruction error and critic output
params = {"rec_error_type": "dtw", "comb": "mult"}

primitive = load_primitive("orion.primitives.tadgan.score_anomalies", 
                           arguments=params)
errors, true_index, true, predictions = primitive.produce(y=y_test, y_hat=y_hat, critic=critic, index=index_test)

errors.shape, true_index.shape

((2276,), (2177,))

In [77]:
# extracts anomalies from sequences of errors following the approach
params = {
    "window_size_portion": 0.33, 
    "window_step_size_portion": 0.1,
    "fixed_threshold": True
}

primitive = load_primitive("orion.primitives.timeseries_anomalies.find_anomalies", 
                           arguments=params)
e = primitive.produce(errors=errors, index=true_index)

e.shape

(1, 3)

In [86]:
predicted_anomalies = [(int(i[0]), int(i[1])) for i in e]
start, end = index_test[0], index_test[-1]
contextual_f1_score(anomalies, predicted_anomalies, start=start, end=end, weighted=True)

0.09674728940783987

In [87]:
0.07401696222050887
0.09674728940783987

0.09674728940783987

In [88]:
predicted_anomalies

[(1150, 1207)]

In [89]:
print(anomalies)

  source  name signal  start   end
0   NASA  SMAP    M-2   1110  2250
